In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk.collocations as collocations
import re
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle

In [2]:
f = open('./movie_reviews/train.tsv', 'r')
#f = open('train.tsv', 'r')
# loop over lines in the file and use the first limit of them
phrasedata = []
for line in f:
  # ignore the first line starting with Phrase and read all lines
  if (not line.startswith('Phrase')):
    # remove final end of line character
    line = line.strip()
    # each line has 4 items separated by tabs
    # ignore the phrase and sentence ids, and keep the phrase and sentiment
    phrasedata.append(line.split('\t')[2:4])
f.close()

In [3]:
import random

In [4]:
random.shuffle(phrasedata)
phraselist = phrasedata
for phrase in phraselist[:10]:
    print(phrase)

['a stifling morality tale', '2']
['exception', '2']
['quaking essence', '2']
['sitcom-worthy solutions', '1']
['plenty of nudity', '3']
['is just too original to be ignored', '3']
["makes Oliver far more interesting than the character 's lines would suggest", '3']
["The movie is like Scorsese 's Mean Streets redone by someone who ignored it in favor of old ` juvenile delinquent ' paperbacks with titles like Leather Warriors and Switchblade Sexpot .", '1']
['might have an opportunity to triumphantly sermonize', '2']
['feel-good , follow-your-dream Hollywood fantasies', '2']


In [5]:
#%pip install nltk

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
phrasedocs = []
for phrase in phraselist:
    tokens = nltk.word_tokenize(phrase[0])
    phrasedocs.append((tokens, int(phrase[1])))

In [8]:
for phrase in phrasedocs[:10]:
    print(phrase)

(['a', 'stifling', 'morality', 'tale'], 2)
(['exception'], 2)
(['quaking', 'essence'], 2)
(['sitcom-worthy', 'solutions'], 1)
(['plenty', 'of', 'nudity'], 3)
(['is', 'just', 'too', 'original', 'to', 'be', 'ignored'], 3)
(['makes', 'Oliver', 'far', 'more', 'interesting', 'than', 'the', 'character', "'s", 'lines', 'would', 'suggest'], 3)
(['The', 'movie', 'is', 'like', 'Scorsese', "'s", 'Mean', 'Streets', 'redone', 'by', 'someone', 'who', 'ignored', 'it', 'in', 'favor', 'of', 'old', '`', 'juvenile', 'delinquent', "'", 'paperbacks', 'with', 'titles', 'like', 'Leather', 'Warriors', 'and', 'Switchblade', 'Sexpot', '.'], 1)
(['might', 'have', 'an', 'opportunity', 'to', 'triumphantly', 'sermonize'], 2)
(['feel-good', ',', 'follow-your-dream', 'Hollywood', 'fantasies'], 2)


In [9]:
docs = []
for phrase in phrasedocs:
    lowerphrase = ([w.lower() for w in phrase[0]], phrase[1])
    docs.append(lowerphrase)
for phrase in docs[:10]:
    print(phrase)

(['a', 'stifling', 'morality', 'tale'], 2)
(['exception'], 2)
(['quaking', 'essence'], 2)
(['sitcom-worthy', 'solutions'], 1)
(['plenty', 'of', 'nudity'], 3)
(['is', 'just', 'too', 'original', 'to', 'be', 'ignored'], 3)
(['makes', 'oliver', 'far', 'more', 'interesting', 'than', 'the', 'character', "'s", 'lines', 'would', 'suggest'], 3)
(['the', 'movie', 'is', 'like', 'scorsese', "'s", 'mean', 'streets', 'redone', 'by', 'someone', 'who', 'ignored', 'it', 'in', 'favor', 'of', 'old', '`', 'juvenile', 'delinquent', "'", 'paperbacks', 'with', 'titles', 'like', 'leather', 'warriors', 'and', 'switchblade', 'sexpot', '.'], 1)
(['might', 'have', 'an', 'opportunity', 'to', 'triumphantly', 'sermonize'], 2)
(['feel-good', ',', 'follow-your-dream', 'hollywood', 'fantasies'], 2)


In [10]:
all_words_list = [word for (sent, cat) in docs for word in sent]
all_words = nltk.FreqDist(all_words_list)
print(len(all_words))

16537


In [11]:
word_items = all_words.most_common(1000)
word_features = [word for (word, count) in word_items]

In [12]:
def document_features(document, word_features):
    document_words = set(document)
    #we open a Pytnon dictionary instead of a list
    features = {}
    for word in word_features:
        #checking if the word from word_features matches a word in the document
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [13]:
len(docs) 

156060

In [14]:
import numpy as np
doc_chunks = np.array_split(np.array(docs), 100)

<ipython-input-14-d1b16b106a7c>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  doc_chunks = np.array_split(np.array(docs), 100)


In [15]:
len(doc_chunks)

100

In [16]:
doc_chunks[0][0][0]

['a', 'stifling', 'morality', 'tale']

In [17]:
docs[0]

(['a', 'stifling', 'morality', 'tale'], 2)

In [18]:
%%time
feature_set_chunks = []
for chunk in doc_chunks:
    feature_set_chunks.append(
        [(document_features(chunk_sent[0], word_features), chunk_sent[1]) 
         for chunk_sent in chunk]
    )

Wall time: 59.3 s


In [19]:
len(feature_set_chunks)

100

In [20]:
feature_set_chunks[0][0]

({'contains(the)': False,
  'contains(,)': False,
  'contains(a)': True,
  'contains(of)': False,
  'contains(and)': False,
  'contains(to)': False,
  'contains(.)': False,
  "contains('s)": False,
  'contains(in)': False,
  'contains(is)': False,
  'contains(that)': False,
  'contains(it)': False,
  'contains(as)': False,
  'contains(with)': False,
  'contains(for)': False,
  'contains(its)': False,
  'contains(film)': False,
  'contains(an)': False,
  'contains(movie)': False,
  'contains(this)': False,
  'contains(but)': False,
  'contains(be)': False,
  'contains(you)': False,
  'contains(on)': False,
  "contains(n't)": False,
  'contains(by)': False,
  'contains(more)': False,
  'contains(his)': False,
  'contains(about)': False,
  'contains(one)': False,
  'contains(``)': False,
  'contains(not)': False,
  'contains(at)': False,
  'contains(or)': False,
  'contains(from)': False,
  'contains(than)': False,
  'contains(--)': False,
  'contains(all)': False,
  'contains(have)': Fal

In [21]:
import itertools
# Check to see if we get all document featrues from original docs variable
len(list(itertools.chain.from_iterable(feature_set_chunks)))

156060

In [22]:
%%time
featuresets = list(itertools.chain.from_iterable(feature_set_chunks))

Wall time: 12 ms


In [23]:
try:
    del doc_chunks
except Exception as e:
    print(e)

In [24]:
try:
    del feature_set_chunks
except Exception as e:
    print(e)

In [25]:
# featuresets = [(document_features(d, word_features), c) for (d, c) in docs]

In [26]:
len(featuresets)

156060

In [27]:
featuresets[0]

({'contains(the)': False,
  'contains(,)': False,
  'contains(a)': True,
  'contains(of)': False,
  'contains(and)': False,
  'contains(to)': False,
  'contains(.)': False,
  "contains('s)": False,
  'contains(in)': False,
  'contains(is)': False,
  'contains(that)': False,
  'contains(it)': False,
  'contains(as)': False,
  'contains(with)': False,
  'contains(for)': False,
  'contains(its)': False,
  'contains(film)': False,
  'contains(an)': False,
  'contains(movie)': False,
  'contains(this)': False,
  'contains(but)': False,
  'contains(be)': False,
  'contains(you)': False,
  'contains(on)': False,
  "contains(n't)": False,
  'contains(by)': False,
  'contains(more)': False,
  'contains(his)': False,
  'contains(about)': False,
  'contains(one)': False,
  'contains(``)': False,
  'contains(not)': False,
  'contains(at)': False,
  'contains(or)': False,
  'contains(from)': False,
  'contains(than)': False,
  'contains(--)': False,
  'contains(all)': False,
  'contains(have)': Fal

In [28]:
import numpy as np
from sklearn.model_selection import KFold

In [29]:
def kfold_naive_bayes(feature_sets, splits=10):
    kf = KFold(n_splits=splits)
    scores = []

    for train, test in kf.split(feature_sets):
        classifier = nltk.NaiveBayesClassifier.train(
            np.array(feature_sets)[train]
        )
        scores.append(
            nltk.classify.accuracy(classifier, 
                                   np.array(feature_sets)[test])
        )
        
    return classifier, scores

In [30]:
def score_model(true_labels, predictions):
    precision = precision_score(true_labels, predictions, average='macro').round(4)
    recall = recall_score(true_labels, predictions, average='macro').round(4)
    f_measure = f1_score(true_labels, predictions, average='macro').round(4)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F Measure: ", f_measure)
    return precision, recall, f_measure

In [31]:
%%time
uni_5fold_results = kfold_naive_bayes(featuresets, splits=5)

Wall time: 23min 38s


In [32]:
import sys
var_mems = []
local_vars = list(locals().items())
for var, obj in local_vars:
    if not var.startswith("_"):
        print(var, np.round(sys.getsizeof(obj) / 1e6, 4))
#         var_mems.append(var, np.round(sys.getsizeof(obj) / 1e6, 4))

In 0.0003
Out 0.0004
get_ipython 0.0001
exit 0.0
quit 0.0
pd 0.0001
nltk 0.0001
plt 0.0001
sns 0.0001
np 0.0001
collocations 0.0001
re 0.0001
KFold 0.0011
precision_score 0.0001
recall_score 0.0001
f1_score 0.0001
pickle 0.0001
f 0.0002
phrasedata 1.3208
line 0.0001
random 0.0001
phraselist 1.3208
phrase 0.0001
stopwords 0.0
phrasedocs 1.3208
tokens 0.0002
docs 1.3208
lowerphrase 0.0001
all_words_list 9.7847
all_words 0.5899
word_items 0.009
word_features 0.009
document_features 0.0001
chunk 0.0001
itertools 0.0001
featuresets 1.2529
kfold_naive_bayes 0.0001
score_model 0.0001
uni_5fold_results 0.0001
sys 0.0001
var_mems 0.0001


In [33]:
#with open('uni_model_5fold.pickle', 'wb') as f:
with open("./models/uni_model_5fold.pickle", 'wb') as f:
    pickle.dump(uni_5fold_results[0], f)

In [34]:
np.mean(uni_5fold_results[1])

0.5479943611431499

In [35]:
%%time
uni_5fold_preds_labs = [(uni_5fold_results[0].classify(features), label) for features, label in featuresets] 

Wall time: 13min 13s


In [36]:
preds = []
trues = []
for pred, true in uni_5fold_preds_labs:
    preds.append(pred)
    trues.append(true)

uni_5fold_preds_labs = [trues, preds]
uni_5fold_scores = score_model(uni_5fold_preds_labs[0], uni_5fold_preds_labs[1])

Precision:  0.4276
Recall:  0.3918
F Measure:  0.3957
